In [83]:
import pandas as pd
import os
import numpy as np
from pandas import DataFrame

train_df = pd.read_csv(os.path.join(os.getcwd(), 'train.csv'))
test_df = pd.read_csv(os.path.join(os.getcwd(), 'test.csv'))

def convert_to_features_and_target(df: DataFrame, is_training_data=True):
  # Create two dataframes: one for all columns except the last one and another for only the last column
  features_df = df.iloc[:, 1:-1] if is_training_data else df.iloc[:, 1:]
  # Convert the features to a two-dimensional array and the target to a one-dimensional array
  features_set = features_df.values
  target_set = None
  # Convert the feature values to a regular float format
  features_set = features_set.astype(float)
  np.set_printoptions(suppress=True)
  
  if is_training_data:
    target_df = df.iloc[:, -1]
    target_set = target_df.values

  return [features_set, target_set]

[features_training_set, target_training_set] = convert_to_features_and_target(train_df, is_training_data=True)
[testing_set, _] = convert_to_features_and_target(test_df, is_training_data=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0)
clf.fit(features_training_set, target_training_set)

# Test with the testing set
print(clf.predict(features_training_set))

[1 1 1 ... 0 0 0]
